# Predict heart disease using Machine Learning

this notebook looks into using various Python-based machine learning and data science libraries to build a machine learning model capable of predicting whether or not someone has heart disease.

We are going to take the following steps:

1. Problem definition
2. Data
3. Evaluation
4. Features
5. Modeling
6. Experimentation

## 1. Problem definition

In a statement,

> Given clinical parameters about a patient, can we predict whether or not they have heart disease?

## 2. Data

The original data came from the Cleavland data from the UCI Machine Learning Repository. [click here](https://archive.ics.uci.edu/ml/datasets/heart+Disease)

## 3. Evaluation

> if we can reach an accuracy greater than 0.9, we'll pursue the project.

## 4. Features

What features that we will use ?
This is where we different information about our dataset

**Create a data dictionary**

1. age - age in years
2. sex - (1 = male; 0 = female)
3. cp - chest pain type
   - 0: Typical angina: chest pain related decrease blood supply to the heart
   - 1: Atypical angina: chest pain not related to heart
   - 2: Non-anginal pain: typically esophageal spasms (non heart related)
   - 3: Asymptomatic: chest pain not showing signs of disease
4. trestbps - resting blood pressure (in mm Hg on admission to the hospital) anything above 130-140 is typically cause for concern
5. chol - serum cholestoral in mg/dl
   - serum = LDL + HDL + .2 \* triglycerides
   - above 200 is cause for concern
6. fbs - (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
   - '>126' mg/dL signals diabetes
7. restecg - resting electrocardiographic results
   - 0: Nothing to note
   - 1: ST-T Wave abnormality can range from mild symptoms to severe problems signals non-normal heart beat
   - 2: Possible or definite left ventricular hypertrophy Enlarged heart's main pumping chamber
8. thalach - maximum heart rate achieved
9. exang - exercise induced angina (1 = yes; 0 = no)
10. oldpeak - ST depression induced by exercise relative to rest looks at stress of heart during excercise unhealthy heart will stress more
11. slope - the slope of the peak exercise ST segment
    - 0: Upsloping: better heart rate with excercise (uncommon)
    - 1: Flatsloping: minimal change (typical healthy heart)
    - 2: Downslopins: signs of unhealthy heart
12. ca - number of major vessels (0-3) colored by flourosopy
    - colored vessel means the doctor can see the blood passing through
    - the more blood movement the better (no clots)
13. thal - thalium stress result
    - 1,3: normal
    - 6: fixed defect: used to be defect but ok now
    - 7: reversable defect: no proper blood movement when excercising
14. target - have disease or not (1=yes, 0=no) (= the predicted attribute)

### Preparing the tools


In [4]:
# Regular EDA(exploratory data analysis) and plotting library

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Model from sklearn

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Data preprocessing
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer
# Model evaluation
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import RocCurveDisplay, confusion_matrix, classification_report, precision_score, recall_score, f1_score


### Load data


In [8]:
df = pd.read_csv("../data/heart.csv")
df.shape

(303, 14)

### Exploratory Data Analysis

We can learn more about our dataset by visualizing the relationship between features and label.

1. What kind of data do we have and how to treat different types?
2. What is missing from the data and how to deal with it?
3. Where are the outliers and why should we care about them?
4. How can we add, change or remove features to get more out of our data?


In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df['target'].value_counts()

In [ ]:
df['target'].value_counts().plot(
    kind='bar',
    color=["red", "lightblue"],
)


In [ ]:
df.describe()

In [ ]:
# Are there missing values?
df.isna().sum()

In [ ]:
### Heart Disease Frequency according to sex

In [ ]:
df.sex.value_counts()

In [ ]:
# Compare target column with sex column
pd.crosstab(df.target, df.sex)

In [ ]:
pd.crosstab(df.target, df.sex).plot(kind="bar", color=["red", "blue"])
plt.title("Heart Disease Frequency for Sex")
plt.legend(["Female", "Male"])
plt.xlabel("0 = No Disease, 1 = Disease")
plt.xticks(rotation=0)
plt.ylabel("Amount")

In [ ]:
df.head()

In [ ]:
df.age.plot(kind="hist", bins=7)

### Age and Max heart rate for heart disease


In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df.age[df.target == 1], df.thalach[df.target == 1], c='salmon')
plt.scatter(df.age[df.target == 0], df.thalach[df.target == 0], c='lightblue')

plt.title("Heart disease in function of Age and Max heart rate")
plt.legend(["Disease", "No Disease"])
plt.show()

### Heart Disease Frequency per Chest Pain Type


In [ ]:
pd.crosstab(df.cp, df.target).plot.bar(color=["b", 'r'])
plt.title("Heart disease frequency per chest pain type")
plt.legend(["No disease", "disease"])
plt.xticks(rotation=0)
plt.ylabel("Frequency")
plt.xlabel("Chest pain type")

In [ ]:
corr_matrix = df.corr()

fig, ax = plt.subplots(figsize=(15, 10)) 
ax = sns.heatmap(corr_matrix, annot=True, cmap='YlGnBu', linewidths=0.5)

## 5. Model

In [ ]:
y= df.target
X = df.drop(["target"], axis=1)

np.random.seed(42)

X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2)

We will train 3 different models:
1. LogisticRegression
2. RandomForestClassifier
3. KNeighborsClassifier

In [ ]:
models = {
    "rdf": RandomForestClassifier(),
    "lgr": LogisticRegression(max_iter=100),
    "knn": KNeighborsClassifier()
}

def fit_and_score(models, X_train, y_train, X_test, y_test):
    """[Evaluate given machine learning models]

    Args:
        models ([List of scikitlearn model]): [Scikitlearn model]
        X_train ([numpy array]): [Training feature]
        y_train ([type]): [training label]
        X_test ([type]): [testing features]
        y_test ([type]): [testing label]
    """
    np.random.seed(42)
    model_scores = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        model_scores[name] = model.score(X_test, y_test)

    return model_scores


model_scores = fit_and_score(models, X_train, y_train, X_test, y_test)
### Model comparison

model_compare = pd.DataFrame(model_scores, index=["accuracy"])
model_compare.T.plot.bar()


## 6. Evaluation

1. Hyperparameter turning
2. Feature important
3. Confusion matrix
4. Cross-validation
5. Precision
6. Recall
7. F1 score
8. Classification Report
9. ROC curve
10. Area under the curve(AUC)

### Hyperparameter turning

In [ ]:
# KNN
train_scores = []
test_scores = []
knn = KNeighborsClassifier()
for i in range(1,25):
    knn.set_params(n_neighbors =i)
    knn.fit(X_train, y_train)
    train_scores.append(knn.score(X_train, y_train))
    test_scores.append(knn.score(X_test, y_test))

plt.plot(range(1,25),train_scores)
plt.plot(range(1,25),test_scores)
plt.xlabel("Number of neighbors")
plt.ylabel("Accuracy")
plt.legend(['Train score', 'Test score'])
 

### Hyperparameter turning with RandomizedSearchCV



In [ ]:
log_regression_grid = {"C": np.logspace(-4, 4, 20), "solver": ['liblinear']}

rf_grid = {
    "n_estimators": np.arange(10, 100, 10),
    "max_depth": [None, 3,  10],
    "min_samples_split": np.arange(2, 20, 4),
    "min_samples_leaf": np.arange(1, 20, 4)
}


In [ ]:
np.random.seed(42)

lr_searchCV = RandomizedSearchCV(LogisticRegression(),
                                 log_regression_grid,
                                 cv=5,
                                 n_iter=20,
                                 verbose=True)
lr_searchCV.fit(X_train, y_train)


In [ ]:
lr_searchCV.best_params_

In [ ]:
lr_searchCV.score(X_test, y_test)

In [ ]:
np.random.seed(42)
rfc_GSCV = GridSearchCV(RandomForestClassifier(), rf_grid, cv=5)

rfc_GSCV.fit(X_train, y_train)


In [ ]:
rfc_GSCV.score(X_test, y_test)

### Evaluate our machine learning model.


In [ ]:
model = lr_searchCV.best_estimator_
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

##### AUC - ROC

In [ ]:
RocCurveDisplay.from_estimator(model, X_test, y_test)



#### Confusion matrix


In [ ]:
print(confusion_matrix(y_test, y_pred))


def plot_confusion_matrix(y_test, y_pred):
    """
    Plot confusion_matrix using seaborn
    """
    fig, ax = plt.subplots(figsize=(3, 3))
    ax = sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, cbar=False)
    plt.xlabel('True label')
    plt.ylabel('Predicted')


plot_confusion_matrix(y_test, y_pred)
 


In [ ]:

print(classification_report(y_test, y_pred))

#### Evalutaion matrix using cross validation

In [ ]:
# Cross-validated accuracy

cv_acc = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print(cv_acc.mean())

print(model.score(X_test, y_test))

In [ ]:
# Cross-validated precision

cv_prec = cross_val_score(model, X, y, cv=5, scoring='precision')
print(cv_prec.mean())


# Cross-validated accuracy

cv_recall = cross_val_score(model, X, y, cv=5, scoring='recall')
print(cv_recall.mean())


# Cross-validated accuracy

cv_f1 = cross_val_score(model, X, y, cv=5, scoring='f1')
print(cv_f1.mean())

### Features important

In [ ]:
from sklearn.feature_selection import SelectFromModel


feature_dictionary = dict(zip(df.columns, model.coef_[0]))
feature_dictionary
feature_dictionary_dataframe = pd.DataFrame(feature_dictionary, index=['Coef'])
feature_dictionary_dataframe.T.plot.bar()

In [ ]:
df.var()/len(df)

In [ ]:
np.random.seed(42)
filter_model= SelectFromModel(model, max_features=12)
filter_model.fit(X,y)
X_filted = filter_model.transform(X)
feature_names = np.array(X.columns)[filter_model.get_support()]
X_filted_values = filter_model.transform(X)
X_filted = pd.DataFrame(X_filted_values, columns =feature_names, index=None)
X_filted
X_train, X_test, y_train, y_test = train_test_split(X_filted,  y, test_size =0.2)
model.fit(X_train, y_train)
model.score(X_test, y_test)

## 7. Experimentation

* Could we collect more data?
* Could we try a better model?
* Could you improve the current model?
* If our model is good enough, how can we export it and share it with other people?